# Advent of Code 2021 - Day 4

In [177]:
typealias Board = List<List<Int>>
data class Bingo(val draws: List<Int>, val boards: List<Board>)

In [178]:
import java.io.File
import java.util.Scanner

val regex = Regex("""(\d+)""")

val bingo: Bingo = Scanner(File("Day4.input.txt"))
    .apply { useDelimiter("${System.lineSeparator()}${System.lineSeparator()}") }
    .use { scanner ->
        generateSequence { if(scanner.hasNext()) scanner.next() else null }
            .toList()
            .let {
                Bingo(
                    draws = regex.findAll(it[0]).map(MatchResult::value).map(String::toInt).toList(),
                    boards = it.drop(1).map { it.split("\n").map { regex.findAll(it).map(MatchResult::value).map(String::toInt).toList() } }
                )
            }
    }

## Part 1

Draw numbers until one of the `Bingo` `boards` win. A win is if all numbers in a board's rows or columns are marked.

In [179]:
class MarkableBoard(val board: Board) {
    private val _marks: MutableMap<Int, Boolean> = board.flatMap { row -> row.map { it to false } }.toMap().toMutableMap()
    val marks: Map<Int, Boolean> = _marks

    var lastMark: Int? = null
        private set

    val rowWinCondition: List<List<() -> Boolean>> = board.map { row -> row.map { { marks[it]!! } } }
    val columnWinCondition: List<List<() -> Boolean>> = (0 until board.size).map { index -> board.map { { marks[it[index]]!! } } }
    
    fun checkWin(): Boolean = (rowWinCondition + columnWinCondition).any { row -> row.all { it() } }

    fun mark(number: Int): Boolean? = 
        if (_marks.containsKey(number)) 
            _marks.put(number, true).also { lastMark = number }
        else 
            false

    fun score(): Int = lastMark?.let { last -> marks.entries.filter { it.value == false }.sumOf { it.key } * last } ?: 0
}

fun Bingo.winner(): MarkableBoard? {
    val markableBoards = bingo.boards.map(::MarkableBoard)

    bingo.draws.forEach { draw ->
        markableBoards.forEach { 
            it.mark(draw) 
            if (it.checkWin()) return it
        }
    }
    
    return null
}

bingo.winner()?.score()

8442

### Notes

This problem was quite fun, I'm pretty happy with the use of lambdas for this solution. This solution is more imperative than my other solutions, but I like that it highlights Kotlin's treatment of functions as first-class citizens.

The idea here is to first build a map that can tell us which numbers have been marked. To check the win conditions, we can save a list of functions that are positionally tied to the appropriate rows / columns that when executed will retrieve their values from the map. Thus, whenever we need to check the winner, each list of win condition functions can fetch their "status." The logic for checking a win is trivial, since we can just check if any one list is all true. 

## Part 2

What is the final score of the last board that will win?

In [180]:
fun Bingo.lastWinner(): MarkableBoard? {
    var markableBoards = bingo.boards.map(::MarkableBoard)

    bingo.draws.forEach { draw ->
        markableBoards = markableBoards.filterNot {
            it.mark(draw)
            val won = it.checkWin()
            if (markableBoards.size == 1 && won) return it else won
        }
    }

    return null
}

bingo.lastWinner()?.score()

4590

### Notes

Satisfying problem! The setup from the last part made this part fairly easy. Again, more imperative than I normally write, but it works!